## Best New Music: Spotify
### Notebook 2
##### Building on the Pitchfork scrape, this notebook loops through the new music entries and adds a Spotify link if one is available
##### Jack Collins

In [25]:
import pandas as pd
import requests
import json
import time
import re
import base64

In [26]:
# Load in the .csv from the last notebook
pitch = pd.read_csv('new_music.csv')

In [27]:
pitch.head()

,artist,title,review_link,album_track,genre,artwork
0,Kali Uchis,Red Moon in Venus,https://pitchfork.com/best//reviews/albums/kal...,Album,['Pop/R&B'],https://media.pitchfork.com/photos/63cece0fa59...
1,Various Artists,Soul’d Out: The Complete Wattstax Collection,https://pitchfork.com/best//reviews/albums/var...,Album,[],https://media.pitchfork.com/photos/63ff7859f53...
2,Model/Actriz,Dogsbody,/reviews/albums/model-actriz-dogsbody/,Album,Rock,https://media.pitchfork.com/photos/63c562f0688...
3,Liv.e,Girl in the Half Pearl,/reviews/albums/live-girl-in-the-half-pearl/,Album,Pop/R&B,https://media.pitchfork.com/photos/63b85a20198...
4,Caroline Polachek,"Desire, I Want to Turn Into You",/reviews/albums/caroline-polachek-desire-i-wan...,Album,Pop/R&B,https://media.pitchfork.com/photos/638e25b9050...


In [31]:
# Spotify client ID and secret
client_id = '69b2947b7d2b4fe8815c6017a2ebadd8'
client_secret = '8fabea4ea9e446b7b9ce96cf74e3b79f'

In [32]:
def get_token():
    """Function to get the API token, just following the Spotify docs basically..."""
    auth_str = client_id + ":" + client_secret
    auth_bytes = auth_str.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    url = "https://accounts.spotify/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    retry_counter = 0
    #result = post(url, headers=headers, data=data, timeout=10)
    while retry_counter < 5:
        try:
            result = requests.post(url, headers=headers, data=data, timeout=5)
        except socket.error as error:
            print(f"Connection Failed due to socket - {error}")
            print(f"Attempting {retry_counter} of 5")
            time.sleep(3)
            retry_counter  += 1
    json_result = json.loads(result.content)
    token = json_results["access_token"]
    return token

In [33]:
token = get_token()
token

Connection Failed due to socket - HTTPSConnectionPool(host='accounts.spotify', port=443): Max retries exceeded with url: /api/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D6BCBFE880>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Attempting 0 of 5
Connection Failed due to socket - HTTPSConnectionPool(host='accounts.spotify', port=443): Max retries exceeded with url: /api/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D6BCBFE610>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Attempting 1 of 5
Connection Failed due to socket - HTTPSConnectionPool(host='accounts.spotify', port=443): Max retries exceeded with url: /api/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D6BCBFEDC0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Attempting 2 of 5
Connection Failed due to socket - 

UnboundLocalError: local variable 'result' referenced before assignment

In [8]:
# Set up your access token endpoint and data
auth_url = 'https://accounts.spotify.com/api/token'
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

In [9]:
# Request an access token
auth_response = requests.post(auth_url, data=auth_data)
auth_response_data = json.loads(auth_response.text)
access_token = auth_response_data['access_token']

In [10]:
# Set up the search endpoint and data
search_url = 'https://api.spotify.com/v1/search'
search_data = {
    'q': 'album:"The Dark Side of the Moon"',
    'type': 'album'
}

In [11]:
# Search for the album and extract its URI
headers = {
    'Authorization': f'Bearer {access_token}'
}
search_response = requests.get(search_url, params=search_data, headers=headers)
search_response_data = json.loads(search_response.text)
album_uri = search_response_data['albums']['items'][0]['uri']

In [12]:
# Extract the playlist ID from the playlist URL
playlist_url = 'https://open.spotify.com/playlist/4CQjOow2bgRcRg9eZJTmBH?si=f79f218505e64b9b'
playlist_id = re.search(r'playlist/(\w+)', playlist_url).group(1)

In [13]:
playlist_id

'4CQjOow2bgRcRg9eZJTmBH'

In [14]:
# Set up the add-to-playlist endpoint and data
playlist_uri = f'spotify:playlist:{playlist_id}'
add_to_playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_uri}/tracks'
add_to_playlist_data = {
    'uris': [album_uri]
}

In [15]:
# Add the album to the playlist
add_to_playlist_response = requests.post(add_to_playlist_url, json=add_to_playlist_data, headers=headers)

In [16]:
add_to_playlist_response

<Response [404]>

In [ ]:
add_to_playlist_url

In [ ]:
# Combining these fields makes it easier to search
pitch['title_artist'] = pitch['artist'] + ' ' + pitch['title']

In [11]:
# Bringing in Spotify credentials
from credentials import s_key, ss_key

ModuleNotFoundError: No module named 'credentials'

In [12]:
#This function gets an auth token
def auth(key, sec):
    
    authorize = 'https://accounts.spotify.com/api/token'
    param = {
    "Content-Type": "application/x-www-form-urlencoded",
    'grant_type' : 'client_credentials'
    }
    res = requests.post(authorize, auth = (s_key, ss_key), data = param)
    token = res.json()['access_token']
    
    return token

In [9]:
#This function loops through the dataframe, search Spotify, and adds the link to listen if one is available

def get_uri(df):
    #Set up empty counter and total songs to get
    count = 0 
    total = len(df)
    token = auth(s_key, ss_key)
    
    for i in range(len(df)):
        count += 1
        song = df.loc[i, 'title_artist']
       
        try:
            
            #Search parameters with the song plugged in
            params = {
            'q' : song,
            'type': 'album',
            'limit' : 5
            }

            #Header for authorization
            header = {'Authorization' : f'Bearer {token}'}

            #Search endpoint
            spotify_search = 'https://api.spotify.com/v1/search'

            #Make the request
            res = requests.get(spotify_search, headers = header, params = params)
            status = res.status_code
            
            #Continue if status code is 200:
            if res.status_code == 200:
            
                #Save the results and grab the uri
                results = res.json()

                uri = results['albums']['items'][0]['external_urls']['spotify']
            
            else:
                print(df.loc[i, 'title_artist'])
        
        except:
            #If it doesn't work, print out the song and fill in with nan
            print(f'Could not retrieve {song}, status code {status}')
            uri = 'nan'
            
        #Add the info the the DataFrame:
       
        df.loc[i, 'spotify'] = uri
        
        #Print updates for every 100
        if count % 100 == 0:
            print(f'{count} gathered out of {total}')
        
        #Take a break to not exceed the rate limit
        time.sleep(1)

    return df

In [10]:
pitch = get_uri(pitch)

NameError: name 's_key' is not defined

In [ ]:
#Returnedf dataframe
pitch